In [1]:
import itertools
import json
import os
import pickle
import re
import string
import tarfile
from collections import Counter, defaultdict
from datetime import datetime
from pprint import pprint
import requests
from math import log2

In [ ]:
res1 = r"(<[^>]*>)"
res2 = r"((-|‒)(\n|\r|\r\n|\x0b|\x0c|\x1c|\x1d|\x1e|\x85|\u2028|\u2029))"
res3 = r"^[a-ząćęłńóśźż]+$"
com1 = re.compile(res1, re.IGNORECASE)
com2 = re.compile(res2, re.IGNORECASE)
pairs1 = re.compile(res3, re.IGNORECASE)

tar = tarfile.open("../saos-dump-23.02.2018.tar.gz", "r:gz",)

In [ ]:
def get_next(tars):
    index = 1
    for tarinfo in tars:
        if tarinfo.isreg():
            if tarinfo.name.startswith('data/json/judgments'):
                pprint(str(index) + ": " + tarinfo.name)
                index += 1
                yield json.load(tars.extractfile(tarinfo)), tarinfo.name[10:]


def given_date(date):
    z = datetime.strptime(date, '%Y-%m-%d')
    return z.year == 2015


def get_pairs(itera):
    first = next(itera)
    for word in itera:
        yield from itertools.product(first, word)
        first = word


def modifi_pairs(ret):
    tmp = []
    for t in filter(lambda y: len(y) > 0, ret.content.decode().split("\n")):
        if t.startswith("\t"):
            r = t.strip().split("\t")
            if pairs1.match(r[0]):
                tmp.append(r[0]+":"+r[1].split(":")[0])
        else:
            if len(tmp) > 0:
                yield tmp
                tmp = []

In [ ]:
def give_me(fun):
    for x, name in get_next(tar):
        for y in x['items']:
            try:
                if given_date(y['judgmentDate']):
                    words = pairs1.sub("", com2.sub("", com1.sub("", y['textContent'])))\
                        .translate(str.maketrans("", "", "§‰‒⚫®÷€●£±" + string.punctuation))
                    ret = requests.post("http://0.0.0.0:9200", data=words.encode())
                    if ret.status_code == 200:
                        path = os.path.join("done", name[:-5] + ":" + str(y["id"]))
                        with open(path, "w") as f:
                            for d in fun(modifi_pairs(ret)):
                                f.write(str(d))
                                yield d
                    else:
                        print(ret.content.decode())
            except KeyError:

                pprint(KeyError)


# two_word = Counter(give_me(get_pairs))
# with open("two_words", "wb") as f:
#     pickle.dump(two_word, f)

In [ ]:
def entropy(*args):
    tmp = sum(args)
    return sum((k * log2(k / tmp + int(k == 0)) for k in args))


def calculate_g2(two_dict: Counter):
    total = sum(two_dict.values())
    table = defaultdict(lambda: Counter())
    table_reverse = defaultdict(lambda: Counter())
    for pair in two_dict:
        table[pair[0]][pair[1]] = two_dict[pair]
        table_reverse[pair[1]][pair[0]] = two_dict[pair]
    for fst, sec in two_dict:
        k11 = table[fst][sec]
        k21 = sum(table[fst].values()) - k11
        k12 = sum(table_reverse[sec].values()) - k11
        k22 = total - k21 - k12 - k11
        yield ((fst, sec, 2 * (entropy(k11, k12, k21, k22) - entropy(k11 + k12, k21 + k22)
                                - entropy(k11 + k21, k12 + k22)))

In [2]:
# with open("two_words_lower", "wb") as f:
#     two_word = Counter(give_me(get_pairs))
#     pickle.dumb(two_words, f)

# with open("two_words_lower", "rb") as f:
#     two_word = Counter(pickle.load(f))
    
# result_g2 = Counter(dict(calculate_g2(two_word)))
# with open("g2_result", "wb") as f:
#     pickle.dump(result_g2, f)

with open("g2_result", "rb") as f:
    result_g2 = Counter(pickle.load(f))

In [3]:
pprint(result_g2.most_common(30))

[(('z:prep', 'dzień:subst'), 4587521.763124168),
 (('kwota:subst', 'złoty:brev'), 3848757.47851724),
 (('sąd:subst', 'okręgowy:adj'), 3537427.676854592),
 (('sąd:subst', 'rejonowy:adj'), 3081445.4619805515),
 (('art:brev', 'kk:brev'), 2436834.728546394),
 (('sygn:subst', 'akt:subst'), 2410002.54784516),
 (('na:prep', 'podstawa:subst'), 2380268.662492305),
 (('ubezpieczenie:subst', 'społeczny:adj'), 2281565.849403266),
 (('skarb:subst', 'państwo:subst'), 2049853.7099823956),
 (('zgodnie:adv', 'z:prep'), 1991863.942045264),
 (('na:prep', 'rzecz:subst'), 1896368.3346092254),
 (('art:brev', 'usta:subst'), 1865540.3137207292),
 (('materiał:subst', 'dowodowy:adj'), 1770403.4192103255),
 (('co:comp', 'do:prep'), 1603840.103305568),
 (('zw:brev', 'z:prep'), 1539189.9207633357),
 (('z:prep', 'art:brev'), 1520207.7367877886),
 (('zeznanie:subst', 'świadek:subst'), 1507013.9391400935),
 (('i:adj', 'instancja:subst'), 1464106.8560040481),
 (('sąd:subst', 'wysoki:adj'), 1435565.2406496778),
 (('org

In [4]:
pprint(list(itertools.islice(filter(lambda x: x[0][0][x[0][0].find(":") + 1:] == "subst" and
                                              (x[0][1][x[0][1].find(":") + 1:] == "subst" or
                                               x[0][1][x[0][1].find(":") + 1:].startwith("adj"),
                                    result_g2.most_common()), 0, 30)))

[(('sąd:subst', 'okręgowy:adj'), 3537427.676854592),
 (('sąd:subst', 'rejonowy:adj'), 3081445.4619805515),
 (('sygn:subst', 'akt:subst'), 2410002.54784516),
 (('ubezpieczenie:subst', 'społeczny:adj'), 2281565.849403266),
 (('skarb:subst', 'państwo:subst'), 2049853.7099823956),
 (('materiał:subst', 'dowodowy:adj'), 1770403.4192103255),
 (('zeznanie:subst', 'świadek:subst'), 1507013.9391400935),
 (('sąd:subst', 'wysoki:adj'), 1435565.2406496778),
 (('organ:subst', 'rentowy:adj'), 1432430.0995914172),
 (('art:subst', 'kc:subst'), 1349819.509069791),
 (('dzień:subst', 'grudzień:subst'), 1196323.1062961742),
 (('sąd:subst', 'apelacyjny:adj'), 1158030.744022239),
 (('dzień:subst', 'wrzesień:subst'), 1130037.8991782926),
 (('koszt:subst', 'proces:subst'), 1079136.9341652086),
 (('rzeczpospolita:subst', 'polski:adj'), 1069153.3114124008),
 (('dzień:subst', 'październik:subst'), 1069130.8697118834),
 (('dzień:subst', 'styczeń:subst'), 1068113.101466693),
 (('dzień:subst', 'czerwiec:subst'), 106